# HORTON 3.x Demo

> **If you are using Google Colab**, you'll need to install packages and download the example files by running the commands in the next cell.

In [ ]:
# Install packages in Google Colab. Don't run this cell if packages/data is already in your environment
! pip install git+https://github.com/theochem/iodata.git
! pip install git+https://github.com/theochem/grid.git
! pip install git+https://github.com/theochem/gbasis.git
! pip install git+https://github.com/theochem/denspart.git
! pip install git+https://github.com/theochem/AtomDB.git@demo_notebook

# download the example files
import os
from urllib.request import urlretrieve

fpath = "data/"
if not os.path.exists(fpath):
    os.makedirs(fpath, exist_ok=True)

urlretrieve(
    "https://raw.githubusercontent.com/theochem/horton3/master/notebooks/data/h2o_sto3g.fchk",
    os.path.join(fpath, "h2o_sto3g.fchk")
    )

# `IOData`

[GitHub Repository](https://github.com/theochem/iodata) | [Webpage](https://iodata.qcdevs.org/) | [Publication](https://onlinelibrary.wiley.com/doi/10.1002/jcc.26468)

> `IOData` is a free and open-source Python library for parsing, storing, and converting various file formats commonly used by quantum chemistry, molecular dynamics, and plane-wave density-functional-theory software programs. It also supports a flexible framework for generating input files for various software packages.

Check [IOData Webpage](https://iodata.qcdevs.org/) for list of supported file formats and properties.

In [ ]:
from iodata import load_one

In [ ]:
mol = load_one("data/h2o_sto3g.fchk")

print("Number of atoms    = ", mol.natom)
print("Atomic numbers     = ", mol.atnums)
print("Atomic coordinates = ")
print(mol.atcoords)
print("")
print("MO Occupations = ", mol.mo.occs)
print("MO Energies    = ", mol.mo.energies)

# `Grid`

[GitHub Repository](https://github.com/theochem/grid) | [Webpage](https://grid.qcdevs.org/)

> `grid` is a free and open-source Python library for numerical integration, interpolation and differentiation of interest for the quantum chemistry community. Grid is intended to support molecular density-functional (DFT) theory calculations, but it also periodic boundary conditions.

In [ ]:
from grid.becke import BeckeWeights
from grid.molgrid import MolGrid
from grid.onedgrid import GaussChebyshev
from grid.rtransform import BeckeRTransform

In [ ]:
# Make molecular grid (using grid package)
oned = GaussChebyshev(100)
rgrid = BeckeRTransform(1e-4, 1.5).transform_1d_grid(oned)
grid = MolGrid.from_size(mol.atnums, mol.atcoords, rgrid, 110, BeckeWeights())

print("Grid points shape  = ", grid.points.shape)
print("Grid weights shape = ", grid.weights.shape)

In [ ]:
# Make molecular grid (using preset grid)
oned = GaussChebyshev(100)
rgrid = BeckeRTransform(1e-4, 1.5).transform_1d_grid(oned)
grid = MolGrid.from_preset(mol.atnums, mol.atcoords, rgrid, "ultrafine", BeckeWeights())

print("Grid points shape  = ", grid.points.shape)
print("Grid weights shape = ", grid.weights.shape)

# `GBasis`

[GitHub Repository](https://github.com/theochem/gbasis)

> `gbasis` is a pure-Python package for evaluating and analytically integrating Gaussian-type orbitals and their related quantities. The goal is to build a set of tools to the quantum chemistry community that are easily accessible and easy to use as to facilitate future scientific works.

In [ ]:
from gbasis.wrappers import from_iodata
from gbasis.evals.density import evaluate_density

import numpy as np   #We need this to evaluate the 1DM from the MOs

In [ ]:
# Compute molecular density (using gbasis package)
one_rdm = np.dot(mol.mo.coeffs * mol.mo.occs, mol.mo.coeffs.T)
basis = from_iodata(mol)
density = evaluate_density(one_rdm, basis[0], grid.points, coord_type=basis[1])

print("Density shape = ", density.shape)

In [ ]:
# Integrate electron density
print("Number of Electrons = ", grid.integrate(density))

# `Denspart`

[GitHub Repository](https://github.com/theochem/denspart)

> Atoms-in-molecules density partitioning schemes for molecules and solids, based on the Hirshfeld/stockholder recipe.

- [Information-Theoretic Atoms in Molecules](https://doi.org/10.1021/acs.jpca.7b08966).
- [Minimal Basis Iterative Stockholder (MBIS)](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00456).

In [ ]:
from denspart.mbis import MBISProModel
from denspart.vh import optimize_reduce_pro_model
from denspart.properties import compute_radial_moments, compute_multipole_moments, safe_ratio

In [ ]:
pro_model_init = MBISProModel.from_geometry(mol.atnums, mol.atcoords)
pro_model, localgrids = optimize_reduce_pro_model(pro_model_init,grid,density)

results = pro_model.to_dict()
results.update({"charges": pro_model.charges,
                 "multipole_moments": compute_multipole_moments(pro_model, grid, density, localgrids)})

print("Atomic Charges    = ", results["charges"])
print("")
print("Atomic Multipoles = ", results["multipole_moments"])

## AtomDB

[GitHub Repository](https://github.com/theochem/AtomDB)

> An Extended Periodic Table of Neutral and Charged Atomic Species

Promolecular approximations
$$
\rho^{\text{extensive}}_{\text{pro}} \approx \sum^{N_{atoms}}_{k=1} w_k \rho_k
$$
$$
\langle x_{\text{intensive}} \rangle_p \approx  \left(\frac{\sum_{k=1}^n w_k x_k^p}{\sum_{k=1}^n w_k} \right)^{\tfrac{1}{p}}
$$

In [ ]:
import atomdb
from atomdb.promolecule import make_promolecule

In [ ]:
# Define species and load data
element = 'O'
charge = 0
mult = 3
dataset = 'slater'

species = atomdb.load(element, charge, mult, dataset=dataset)


# Periodic Table information
print("Number of electrons    = ", species.nelec)
print("Atomic mass    = ", species.mass)
print("MO Occupations = ", species.ao.occs_a + species.ao.occs_b)
print("MO Energies    = ", species.ao.energy_a)
print("Atomic radius [a.u.]    = ", species.vdw_radii['bondi'])

In [ ]:
promol = make_promolecule(mol.atnums,mol.atcoords,dataset=dataset)
print("")
print("Extensive properties:")
print("Atomic mass    = ", promol.mass())
print("Electronic energy = ", -promol.energy())
promol_dens = promol.density(grid.points, log=True)
# Integrate electron density
print("Integral Density  = ", grid.integrate(promol_dens))

promol = make_promolecule(mol.atnums, mol.atcoords, charges=results["charges"], dataset='nist')
print("")
print("Intensive properties:")
print("Ionization potential [eV]  = ", promol.ip())
print("Chemical potential [eV]  = ", promol.mu())
print("Chemical hardness [eV]  = ", promol.eta())